In [41]:
using Catlab
using MacroTools
import MacroTools: postwalk, striplines

In [42]:
using Catlab.WiringDiagrams
using Catlab.Doctrines
using Test
import Catlab.Doctrines.⊗
⊗(a::WiringDiagram, b::WiringDiagram) = otimes(a,b)
import Base: ∘
∘(a::WiringDiagram, b::WiringDiagram) = compose(b, a)
⊚(a,b) = b ∘ a

⊚ (generic function with 1 method)

In [43]:
?WiringDiagram

search: WiringDiagram WiringDiagrams WiringDiagramCore WiringDiagramAlgorithms



Wiring diagram: morphism in the category of wiring diagrams.

The wiring diagram is implemented using the following internal data structures. A LightGraphs `DiGraph` stores the "skeleton" of the diagram: a simple directed graph with the boxes as vertices and with an edge between two vertices iff there is at least one wire between the corresponding boxes. There are two special vertices, accessible via `input_id` and `output_id`, representing the input and output ports, respectively.

The `DiGraph` is wrapped inside a `MetaDiGraph` to attach properties to the vertices and edges. For each edge, an edge property stores the list of wires between the source and target boxes.

---

Create empty wiring diagram with given domain and codomain objects.

---

Create wiring diagram with a single box containing a morphism expression.


In [44]:
# Generators
A, B, C, D = Ob(FreeSymmetricMonoidalCategory, :A, :B, :C, :D)
f = WiringDiagram(Hom(:f,A,B))
g = WiringDiagram(Hom(:g,B,A))


@test nboxes(f) == 1
@test boxes(f) == [ Box(Hom(:f,A,B)) ]
@test nwires(f) == 2
@test Ports([:A]) == Ports([:A])
@test WiringDiagram(Hom(:f,A,B)) == WiringDiagram(Hom(:f,A,B))

Test Passed

In [45]:
Hom(:h, otimes(A,B), C)

h

In [46]:
Hom(:f, A, B) ⊗ Hom(:g, B, C)

otimes(f,g)

In [47]:
Hom(:g, B, C) ∘ Hom(:f, A, B)

compose(f,g)

In [48]:
@test_throws SyntaxDomainError Hom(:g, B, C) ⊚ Hom(:f, A, B) 

Test Passed
      Thrown: SyntaxDomainError

In [49]:
@test compose(Hom(:f, A, B), Hom(:g, B, C)) == ⊚(Hom(:f, A,B), Hom(:g, B, C))

Test Passed

In [50]:
h = WiringDiagram(Hom(:h, otimes(A,B), C)) 

WiringDiagram([:A,:B], [:C], 
[ 1 => {inputs},
  2 => {outputs},
  3 => Box(:h, [:A,:B], [:C]) ],
[ Wire((1,1) => (3,1)),
  Wire((1,2) => (3,2)),
  Wire((3,1) => (2,1)) ])

In [51]:
@test compose(otimes(f,g), h) == (f⊗g) ⊚ h

Test Passed

Scientists will draw diagrams that capture the behavior of physical systems. These diagrams are often structured according to what category theorist call a *Symmetric Monoidal Category* these diagrams have wires that represent *things* and boxes that represent *events*, and there are rules for composing little diagrams to make big ones.

1. If the output wires of A are the same type as the input wires of B, you can combine them in series to make $B \circ A$.
2. If the A and B are diagrams, you can combine them in parallel them to make $A\otimes B$
3. If two wires cross, you can un cross them.

Rule 1 is composition, Rule 2 is the monoidal product $\otimes$ these diagrams arose from Feynman Diagrams where the "in parallel" rule is tensor product of vector spaces. And Rule 3 is the symetry condition. These are abbreviated SMC for *Categorie, Monoidal, Symetrique*

An SMC is called *finitely generated* if there are a finite number of objects (wires) and events (morphisms) that can be composed freely.

In [52]:
# Generators
S, E, I, R, D= Ob(FreeSymmetricMonoidalCategory, :S, :E, :I, :R, :D)

5-element Array{Catlab.Doctrines.FreeSymmetricMonoidalCategory.Ob{:generator},1}:
 S
 E
 I
 R
 D

We are going to model some infectious processes in epidemiology as wiring diagrams and then show how to convert these process models into differential equations and solve them.

In [53]:
# A person could get sick spontaniously
ens = Hom(:ensicken, S, I)

ensicken

In [54]:
#And sick people recover spontaneously too.
recovery = Hom(:recover, I, R)

recover

In [55]:
# We can compose these to make a diagram of illness
compose(WiringDiagram(ens), WiringDiagram(recovery))

WiringDiagram([:S], [:R], 
[ 1 => {inputs},
  2 => {outputs},
  3 => Box(:ensicken, [:S], [:I]),
  4 => Box(:recover, [:I], [:R]) ],
[ Wire((1,1) => (3,1)),
  Wire((3,1) => (4,1)),
  Wire((4,1) => (2,1)) ])

But that isn't how people get sick usually, they pick it up from someone else who has already been infected. The monoidal product for our case is going to be $A \otimes B$ means a person in state A and a person in state B as an unordered collection so that $A\otimes B = B\otimes A$ and $A\otimes B\otimes C$ is an A, a B, and a C in any order.

In [56]:
infecting = Hom(:infection,S ⊗ I, I⊗I)

infection

In [57]:
inf = WiringDiagram(infecting)
rec = WiringDiagram(Hom(:recovery,I, R))

WiringDiagram([:I], [:R], 
[ 1 => {inputs},
  2 => {outputs},
  3 => Box(:recovery, [:I], [:R]) ],
[ Wire((1,1) => (3,1)),
  Wire((3,1) => (2,1)) ])

We are going to use $\circ$ to mean mathematical composition in the normal ordering $f\circ g (x)= g(f(x))$ and $f \circledcirc g = g(f)$ for the ordering that makes more sense 

If you try to compose these diagrams you will get a domain error because we have $S \otimes I \rightarrow I\otimes I \circledcirc I\rightarrow R$ which doesn't satisfy the consistency requirement of function composition that is $A\rightarrow B \circledcirc B \rightarrow C = A\rightarrow C$.  


In [58]:
SIR = inf ⊚ rec

ErrorException: Incompatible domains Ports{Symbol}(Symbol[:I, :I]) and Ports{Symbol}(Symbol[:I])

We can introduce another diagram to our set of generators to fix it.

In [59]:
SIR = inf ⊚ WiringDiagram(Hom(:recover, I⊗I, R⊗R))

WiringDiagram([:S,:I], [:R,:R], 
[ 1 => {inputs},
  2 => {outputs},
  3 => Box(:infection, [:S,:I], [:I,:I]),
  4 => Box(:recover, [:I,:I], [:R,:R]) ],
[ Wire((1,1) => (3,1)),
  Wire((1,2) => (3,2)),
  Wire((3,1) => (4,1)),
  Wire((3,2) => (4,2)),
  Wire((4,1) => (2,1)),
  Wire((4,2) => (2,2)) ])

How would you interperet the diagram on the right above?

It is a single event that takes to sick people and creates two healthy people. What medical mystery is this? This doesn't correspond to our understanding of the domain. There is no epidemiological meaning to the concept of `Hom(:recover, I⊗I, R⊗R)`. Recovery is an event that happens to individuals, not pairs of individuals, but we if we have two people that need to recover, we can have them recover in parallel.

In [60]:
SIR = inf ⊚ (rec ⊗ rec)

WiringDiagram([:S,:I], [:R,:R], 
[ 1 => {inputs},
  2 => {outputs},
  3 => Box(:recovery, [:I], [:R]),
  4 => Box(:recovery, [:I], [:R]),
  5 => Box(:infection, [:S,:I], [:I,:I]) ],
[ Wire((1,1) => (5,1)),
  Wire((1,2) => (5,2)),
  Wire((3,1) => (2,1)),
  Wire((4,1) => (2,2)),
  Wire((5,1) => (3,1)),
  Wire((5,2) => (4,1)) ])

What we have seen so far represents a disease where you get catch the disease, becaoming infectious, and then recover. That makes sense for chicken pox. 
But what about diseases like the common cold, where you can get the disease again? Our example of the SEIR process can be composed with the waning of immunity $wan: R \rightarrow S$. 

In [61]:
wan = WiringDiagram(Hom(:waning,R, S))
SIRS = SIR ⊚ (wan⊗wan)

WiringDiagram([:S,:I], [:S,:S], 
[ 1 => {inputs},
  2 => {outputs},
  3 => Box(:waning, [:R], [:S]),
  4 => Box(:waning, [:R], [:S]),
  5 => Box(:recovery, [:I], [:R]),
  6 => Box(:recovery, [:I], [:R]),
  7 => Box(:infection, [:S,:I], [:I,:I]) ],
[ Wire((1,1) => (7,1)),
  Wire((1,2) => (7,2)),
  Wire((3,1) => (2,1)),
  Wire((4,1) => (2,2)),
  Wire((5,1) => (3,1)),
  Wire((6,1) => (4,1)),
  Wire((7,1) => (5,1)),
  Wire((7,2) => (6,1)) ])

When combining diagrams, composition only checks that that inputs and outputs of the things you are composing match. These are called *Domain* and *Codomain*. We can see that the SIR model is a map from $S\otimes I \rightarrow R\otimes R$. Which means it takes two sucesptible people to two recovered people. The SIRS model is $SIRS: S\otimes I \rightarrow S\otimes S$ Which means we could compose the SIRS model with another model that expected two suceptible people.

In [62]:
dom(SIR), codom(SIR)

(Ports{Symbol}(Symbol[:S, :I]), Ports{Symbol}(Symbol[:R, :R]))

In [63]:
dom(SIRS), codom(SIRS)

(Ports{Symbol}(Symbol[:S, :I]), Ports{Symbol}(Symbol[:S, :S]))

Lets add a new concept to our SMC representation of the disease. Diseases like the flu have an exposed state, where you can be exposed but not yet infectious.

In [64]:
exposure = WiringDiagram(Hom(:exposure, S⊗I, E⊗I))
exposure ⊚ WiringDiagram(Hom(:progression, E⊗I, I)) ⊚ rec 

WiringDiagram([:S,:I], [:R], 
[ 1 => {inputs},
  2 => {outputs},
  3 => Box(:progression, [:E,:I], [:I]),
  4 => Box(:recovery, [:I], [:R]),
  5 => Box(:exposure, [:S,:I], [:E,:I]) ],
[ Wire((1,1) => (5,1)),
  Wire((1,2) => (5,2)),
  Wire((3,1) => (4,1)),
  Wire((4,1) => (2,1)),
  Wire((5,1) => (3,1)),
  Wire((5,2) => (3,2)) ])

The previous string diagram introduces an element $progression$ that represents the progression of an exposed person into the infectious stage of the disease. This progression element that we introduced is not consitant with the domain understanding of the scientist. As defined above, progression  keeps track of the pairs $S\otimes I \rightarrow E\otimes I$ which represents the person who exposed you. In most diseases, the infectious pathogen is homogeneous meaning that it does not matter who infected you. In this case, it is not meaningful to define progression as taking exposure pairs together.

We want to draw a diagram that doesn't remember these exposure pairs. 

Diagramatic reasoning is like static typing in programming languages. To draw a diagram, we need to match up the domains and codomains of the boxes. This is why the rules of SMC are so useful for modeling processes, we need the ingredients to match the products of each component. This gives us a finite number of ways to combine finitely generated systems. A human designer can solve the SMC compositional constraints in their head and thus generate candidate wiring diagrams.

(Footnote: If you were studying the evolution of a virus and wanted to track these exposure lineages you would have this concept in your domain ontology.)

The Diagram $(E\rightarrow I) \otimes (I\rightarrow R)$ says that the progression from E to I is independent of the recovery from infected to recovered status. This captures our domain ontology concepts that these phenomena are independent, but combine using the tensor product of "happen independently" in a compositional way.

by tensoring the recovery and waning processes and then composing with the exposuring process, we get a model that forgets the pairing between the person who exposed you.

$S\otimes I \rightarrow E\otimes I \rightarrow (E \otimes I \rightarrow I \otimes R)$
which is equivalent to 

$S\otimes I \rightarrow I \otimes R$ or are pair of (suceptible, infected) goes to a pair (infected, recovered). Because the category is monoidal, the order doesn't matter so this model is black-box equivalent to $(S\rightarrow I) \otimes (I \rightarrow R)$. The SMC does not track the identity of the 

In [65]:
seir = exposure ⊚ (WiringDiagram(Hom(:progression, E, I)) ⊗ rec)

WiringDiagram([:S,:I], [:I,:R], 
[ 1 => {inputs},
  2 => {outputs},
  3 => Box(:progression, [:E], [:I]),
  4 => Box(:recovery, [:I], [:R]),
  5 => Box(:exposure, [:S,:I], [:E,:I]) ],
[ Wire((1,1) => (5,1)),
  Wire((1,2) => (5,2)),
  Wire((3,1) => (2,1)),
  Wire((4,1) => (2,2)),
  Wire((5,1) => (3,1)),
  Wire((5,2) => (4,1)) ])

In [66]:
dom(seir), codom(seir)

(Ports{Symbol}(Symbol[:S, :I]), Ports{Symbol}(Symbol[:I, :R]))

If you compose diagrams in the wrong order, the software tells you about the mistake. These checking rules are easy to understand and easy to implement. Which makes them great for representing models. 

In [67]:
seir ⊚ wan

ErrorException: Incompatible domains Ports{Symbol}(Symbol[:I, :R]) and Ports{Symbol}(Symbol[:R])

In [68]:
# the types didn't match, we need to do something with that infected wire
seirs = seir ⊚ (rec⊗wan)

WiringDiagram([:S,:I], [:R,:S], 
[ 1 => {inputs},
  2 => {outputs},
  3 => Box(:recovery, [:I], [:R]),
  4 => Box(:waning, [:R], [:S]),
  5 => Box(:progression, [:E], [:I]),
  6 => Box(:recovery, [:I], [:R]),
  7 => Box(:exposure, [:S,:I], [:E,:I]) ],
[ Wire((1,1) => (7,1)),
  Wire((1,2) => (7,2)),
  Wire((3,1) => (2,1)),
  Wire((4,1) => (2,2)),
  Wire((5,1) => (3,1)),
  Wire((6,1) => (4,1)),
  Wire((7,1) => (5,1)),
  Wire((7,2) => (6,1)) ])

This process is capturing a determinstic infectious disease where each event happens between a pair of people with certainty in a synchronous way . We are able to describe the process of exposure, progression, recovery, and waning of immunity in a deterministic and chronological language, but then translate this string diagram into a petri net that captures the probabilistic nature of compartmental epidemiology models.

String diagrams allow scientists to describe their models in the process model, then functorially map them to categories that capture the dynamics of populations undergoing the process in bulk. We can blackbox those categories into differential equations or stochastic agent based models in order to actually calculate the answers to questions about these systems.

(Footnote: SMC categories allow you to slide boxes along wires to deform the temporal ordering so even though the description we gave is synchronous, you can reorder events that don't share wires.)

In order to use SMC string diagrams as a modeling framework in the context of SemanticModels, we need to write a lens between string diagrams and Julia programs.

In [69]:
#what is a diagram? 
boxes(seirs)

5-element Array{AbstractBox,1}:
 Box(:recovery, [:I], [:R])      
 Box(:waning, [:R], [:S])        
 Box(:progression, [:E], [:I])   
 Box(:recovery, [:I], [:R])      
 Box(:exposure, [:S,:I], [:E,:I])

In [70]:
wirenames(d::WiringDiagram) = foldr(union,
    map(box->union(input_ports(box), output_ports(box)),
        boxes(d)))
function fluxes(d::WiringDiagram)
    # TODO design Multiple Dispatch Lens API
    # TODO use ModelingToolkit variables
    nb = nboxes(d)
    vars = wirenames(d)
    byvar = Dict{Symbol, Expr}()
    homnames = Vector{Symbol}()
    for var in vars
        byvar[var] = :(+())
    end
    map(enumerate(boxes(d))) do (i, box)
        invars = input_ports(box)
        outvars = output_ports(box)
        homname = box.value
        push!(homnames, homname)
        βᵢ = :(p.$(homname))
        ϕ =  :(*($βᵢ, $(invars...)))
        map(invars) do v
            push!(byvar[v].args, :(-$ϕ))
        end
        map(outvars) do v
            push!(byvar[v].args, :($ϕ))
        end
    end
    return byvar, vars, homnames
end
#end

fluxes (generic function with 1 method)

In [71]:
sir = WiringDiagram(Hom(:infection, S⊗I, I⊗I)) ⊚ (rec ⊗ rec)
seir = WiringDiagram(Hom(:exposure, S⊗I, E⊗I)) ⊚ (rec ⊗ WiringDiagram(Hom(:progression, E, I)))
dudt = fluxes(sir)[1]

Dict{Symbol,Expr} with 3 entries:
  :I => :(-(p.recovery * I) + -(p.recovery * I) + -(p.infection * S * I) + p.in…
  :R => :(p.recovery * I + p.recovery * I)
  :S => :(+(-(p.infection * S * I)))

In [72]:
fluxes(seir)[1]

Dict{Symbol,Expr} with 4 entries:
  :I => :(-(p.recovery * I) + p.progression * E + -(p.exposure * S * I) + p.exp…
  :R => :(+(p.recovery * I))
  :S => :(+(-(p.exposure * S * I)))
  :E => :(-(p.progression * E) + p.exposure * S * I)

In [73]:
@show dudt[1][:I]
map(enumerate(dudt[1])) do (i, ϕ)
    :(du.$(ϕ.first) = $(ϕ.second))
end

KeyError: KeyError: key 1 not found

In [74]:
function oderhs(dudt::Dict{T, Expr}, vars::Vector{T}, homnames::Any) where {T}
    lines = map(enumerate(vars)) do (i, v)
        ϕ = dudt[v]
        #:(du.$(ϕ.first) = $(ϕ.second))
        :(du[$(i)] = $(ϕ))
    end
    fdef = quote
    function f(du, u, p, t)
        $(lines...)
        return du
    end
    end
    return fdef, vars, homnames
end

"""    oderhs(d::WiringDiagram)

convert a wiring diagram into a dynamical system described as julia Exprs.
We need to keep track of the names we give the variables and the parameters so those are the second and third arguments respectively.

Returns an expression that defines a function, a list of symbols representing the variables, and a list of symbols representing the parameter names.

see also fluxes(d::WiringDiagram) for implementation details
"""
oderhs(d::WiringDiagram) = oderhs(fluxes(d)...)

oderhs

In [75]:
oderhs(sir)

(quote
    #= In[74]:8 =#
    function f(du, u, p, t)
        #= In[74]:9 =#
        du[1] = -(p.recovery * I) + -(p.recovery * I) + -(p.infection * S * I) + p.infection * S * I + p.infection * S * I
        du[2] = p.recovery * I + p.recovery * I
        du[3] = +(-(p.infection * S * I))
        #= In[74]:10 =#
        return du
    end
end, Symbol[:I, :R, :S], Symbol[:recovery, :recovery, :infection])

In [76]:
"""    odeProblem(d::WiringDiagram, params, initials, tdomain, alg=missing)

bind a wiring diagram to a set of paramters, initial conditions, time domain and solver algorithm and generate the code the solves it.

see also odetemplate.
"""
function odeProblem(d::WiringDiagram, params, initials, tdomain, alg=missing)
    quote
        using DifferentialEquations
        $(oderhs(d)[1])
        function main()
            params = $params
            initials = $initials
            tdomain = $tdomain
            prob = ODEProblem(f, params, initials, tdomain)
            soln = solve(prob, alg=$alg)
            return prob, soln
        end
    end
end


odeProblem

In [77]:
odeProblem(seir, :β, :i₀, (0, 365), :(Tsit5()))

quote
    #= In[76]:9 =#
    using DifferentialEquations
    #= In[76]:10 =#
    begin
        #= In[74]:8 =#
        function f(du, u, p, t)
            #= In[74]:9 =#
            du[1] = -(p.recovery * I) + p.progression * E + -(p.exposure * S * I) + p.exposure * S * I
            du[2] = +(p.recovery * I)
            du[3] = -(p.progression * E) + p.exposure * S * I
            du[4] = +(-(p.exposure * S * I))
            #= In[74]:10 =#
            return du
        end
    end
    #= In[76]:11 =#
    function main()
        #= In[76]:12 =#
        params = β
        #= In[76]:13 =#
        initials = i₀
        #= In[76]:14 =#
        tdomain = (0, 365)
        #= In[76]:15 =#
        prob = ODEProblem(f, params, initials, tdomain)
        #= In[76]:16 =#
        soln = solve(prob, alg=Tsit5())
        #= In[76]:17 =#
        return (prob, soln)
    end
end

In [78]:
"""    odetemplate(d::WiringDiagram)

create an expression that defines a code that solves the ODE.
Given just the wiring diagram, we don't know the paramters, initial conditions, or timedomain, 
so they are passed in as arguments to the function we generate.

These parameters and initial conditions are destructured in the main function so you can
see what the code is expecting to receive by reading the generated output.

The structure of the timedomain is not implied by the wiring diagram so it is passed directly to the
ODEProblem constructor. Any keyword arguments you pass to `main()` are forwarded to `solve()`. 

"""
function odetemplate(d::WiringDiagram)
    f, vars, homnames = oderhs(d)
    params = Expr(:tuple, map(enumerate(homnames)) do (i, name)
        Expr(:(=), name, :(β[$i]))
            end...)
    vars = wirenames(d)
    initials = Expr(:vect, map(vars) do v
            :(i₀.$v)
            end...)
    quote
        using DifferentialEquations
        $(f.args[end])
        function main(β, i₀, tdomain; kwargs...)
            params = $params
            initials = $initials
            prob = ODEProblem(f, params, initials, tdomain)
            soln = solve(prob; kwargs...)
            return prob, soln
        end
    end
end

odetemplate(seir) |> x -> postwalk(x) do x
    return striplines(x)
end

quote
    using DifferentialEquations
    function f(du, u, p, t)
        du[1] = -(p.recovery * I) + p.progression * E + -(p.exposure * S * I) + p.exposure * S * I
        du[2] = +(p.recovery * I)
        du[3] = -(p.progression * E) + p.exposure * S * I
        du[4] = +(-(p.exposure * S * I))
        return du
    end
    function main(β, i₀, tdomain; kwargs...)
        params = (recovery = β[1], progression = β[2], exposure = β[3])
        initials = [i₀.I, i₀.R, i₀.E, i₀.S]
        prob = ODEProblem(f, params, initials, tdomain)
        soln = solve(prob; kwargs...)
        return (prob, soln)
    end
end

In [79]:
# Generators
S, E, I, R, D= Ob(FreeSymmetricMonoidalCategory, :S, :E, :I, :R, :D)

infecting = Hom(:infection, S ⊗ I, I⊗I)

inf  = WiringDiagram(infecting)
expo = WiringDiagram(Hom(:exposure, S⊗I, E⊗I))
rec  = WiringDiagram(Hom(:recovery, I,   R))
wan  = WiringDiagram(Hom(:waning,   R,   S))

si    = WiringDiagram(Hom(:infection,   S⊗I, I⊗I))
se    = WiringDiagram(Hom(:exposure,    S⊗I, E⊗I))
prog  = WiringDiagram(Hom(:progression, E,   I))
fatal = WiringDiagram(Hom(:die,  I, D))
rip   = WiringDiagram(Hom(:rest, D, D))

sir    = si    ⊚ (rec   ⊗ rec)
seir   = se    ⊚ (prog  ⊗ rec)
seirs  = seir  ⊚ (wan   ⊗ wan)
seird  = seir  ⊚ (fatal ⊗ WiringDiagram(Hom(:id, R, R)))
seirds = seird ⊚ (rip   ⊗ wan)

odetemplate(seirds) |> x -> postwalk(x) do x
    return striplines(x)
end

quote
    using DifferentialEquations
    function f(du, u, p, t)
        du[1] = -(p.rest * D) + p.rest * D + p.die * I
        du[2] = -(p.waning * R) + -(p.id * R) + p.id * R + p.recovery * I
        du[3] = p.waning * R + -(p.exposure * S * I)
        du[4] = -(p.die * I) + p.progression * E + -(p.recovery * I) + -(p.exposure * S * I) + p.exposure * S * I
        du[5] = -(p.progression * E) + p.exposure * S * I
        return du
    end
    function main(β, i₀, tdomain; kwargs...)
        params = (rest = β[1], waning = β[2], die = β[3], id = β[4], progression = β[5], recovery = β[6], exposure = β[7])
        initials = [i₀.D, i₀.R, i₀.S, i₀.I, i₀.E]
        prob = ODEProblem(f, params, initials, tdomain)
        soln = solve(prob; kwargs...)
        return (prob, soln)
    end
end